In [2]:
import os, pandas as pd, numpy as np
from sklearn.model_selection import train_test_split

# 基础
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shutil
import os
import time
from tqdm import tqdm_notebook as tqdm
import pickle
import gc

# 绘图工具
import seaborn as sns
import matplotlib.pyplot as plt

# torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# transformers
# from transformers import BertTokenizer, BertModel, BertConfig

# import pytorch_lightning as pl

data_path = '../../data/MovieLens2018-large/'

In [3]:
# # 存储
X_train1 = pd.read_pickle(data_path+'X_train1.pkl')
X_test1= pd.read_pickle(data_path+'X_test1.pkl')
y_train= pd.read_pickle(data_path+'y_train.pkl')
y_test= pd.read_pickle(data_path+'y_test.pkl')

In [5]:
X_test1.head(5)

,userId,movieId,movie_belong_to_(no genres listed),movie_belong_to_Action,movie_belong_to_Adventure,movie_belong_to_Animation,movie_belong_to_Children,movie_belong_to_Comedy,movie_belong_to_Crime,movie_belong_to_Documentary,...,IMAX_by_movie,Musical_by_movie,Mystery_by_movie,Romance_by_movie,Sci-Fi_by_movie,Thriller_by_movie,War_by_movie,Western_by_movie,rating_user_mean,rating_movie_mean
1274695,70515,95873,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,4.325581,0.0,0.000000,0.0,0.000000,3.454545,3.721591
1402278,244402,128360,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,4.315261,4.555556,3.795685
1753924,247757,5064,0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,4.393333,0.0,0.000000,3.190476,3.763787
1887606,187563,1518,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.000000,0.0,4.263158,0.0,0.000000,3.525000,3.373646
93672,29205,111,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.000000,0.0,4.491210,0.0,0.000000,3.109375,4.084811


In [53]:
class GlobalConfig():
    
    def __init__(self):
        self.name = 'v2'
        self.batch_size = 400
        self.epochs = 50
#         self.max_token_size = 20
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         self.model_name = 'bert-base-cased'
        self.hidden_size = 5
        self.feature_dim = X_test1.shape[1]-2 # drop user and movie id 

CONFIG = GlobalConfig()

In [54]:
class MyDataset(Dataset):
    
    def __init__(self, data):
        self.data = data
        self.x = torch.tensor(self.data.drop(['userId','movieId','label'], axis=1).values).float()
        self.y = torch.tensor(self.data.loc[:,'label'].values).long()

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.x[index], self.y[index]


class MyDataModule():
    
    def __init__(self, Config, data_train, data_test):#, data_eval):
        self.batch_size = Config.batch_size
        self.data_train = data_train
        self.data_test = data_test
#         self.data_eval = data_eval
        
        self.train_dataset = MyDataset(self.data_train)
        self.test_dataset = MyDataset(self.data_test)
#         self.eval_dataset = MyDataset(self.data_eval)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=True
        )
    
#     def eval_dataloader(self):
#         return DataLoader(
#             self.eval_dataset,
#             batch_size=self.batch_size,
#             shuffle=True
#         )

In [55]:
data_module = MyDataModule(
    Config=CONFIG,
    data_train=pd.concat([X_train1.astype(np.float32).fillna(0), y_train.astype(np.float32)], axis=1), 
    data_test=pd.concat([X_test1.astype(np.float32).fillna(0), y_test.astype(np.float32)], axis=1)
)

In [56]:
tmp = data_module.train_dataloader()
for i, j in enumerate(tmp):
    if i == 0: 
        tmp1 = j
        break
print(tmp1[0], tmp1[1])

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.6818, 3.8081],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.3333, 3.7050],
        [0.0000, 1.0000, 1.0000,  ..., 0.0000, 2.6462, 2.8691],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 3.0398, 3.9359],
        [0.0000, 1.0000, 1.0000,  ..., 0.0000, 3.4783, 3.2332],
        [0.0000, 1.0000, 1.0000,  ..., 4.5193, 3.5208, 4.1273]]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [57]:
# 模拟输入数据、参数类型、输出、损失函数
p_tmp = nn.Parameter(torch.zeros([62,1]))
res_tmp = torch.mm(tmp1[0], p_tmp)
# res_tmp

# 模型结构 FM

In [58]:
class FM(nn.Module):

    def __init__(self, latent_dim, feature_dim):
        """

        :param latent_dim: FM 的v，作为交互的隐向量
        :param feature_dim: 特征维度 x [batch_size, feature_num]
        """
        super(FM, self).__init__()

        self.latent_dim = latent_dim
        self.feature_dim = feature_dim

        self.w0 = nn.Parameter(torch.zeros(1,))
        self.w1 = nn.Parameter(torch.zeros([self.feature_dim, 1]))
        self.w2 = nn.Parameter(torch.zeros([self.feature_dim, self.latent_dim]))

    def forward(self, inputs):
        x = inputs
        first_order = self.w0 + torch.mm(x, self.w1) # shape = [batch_size, 1]
        second_order = 1 / 2 * torch.sum(
            torch.pow(torch.mm(x, self.w2), 2) - torch.mm(torch.pow(x, 2), torch.pow(self.w2, 2)),
            dim=1,
            keepdim=True
        ) # shape = [batch_size, 1]
        res = first_order + second_order
        res = torch.sigmoid(res)
        return res

In [59]:
fm = FM(CONFIG.hidden_size, CONFIG.feature_dim)
print(fm.w0.size(), fm.w1.size(), fm.w2.size())
optimizer = optim.Adam(fm.parameters(), lr=0.0001)

torch.Size([1]) torch.Size([62, 1]) torch.Size([62, 5])


In [60]:
# torch.cuda.set_device(0)
testing = tmp1[0]
testing = testing.to(CONFIG.device)
testing.cuda()
fm.cuda()
res_tmp = fm(testing)
print(res_tmp.size())

torch.Size([400, 1])


In [61]:
for i in fm.parameters():
    print(i.size())

torch.Size([1])
torch.Size([62, 1])
torch.Size([62, 5])


## 训练函数
出现了模型偏向于预测0的情况（1的稀疏性），因此要纳入损失权重。

In [62]:
# 手动计算验证权重
import collections, sklearn
label_all = y_train.copy()
t=collections.Counter(label_all)
res = []
for k in t:
    res.append((k, len(label_all)/2/t[k]))
print(res)

class_weights=sklearn.utils.class_weight.compute_class_weight(
    class_weight='balanced',classes=np.unique(label_all),y=label_all)
class_weights=torch.tensor(class_weights,dtype=torch.float)
class_weights

[(0, 0.5858192828876997), (1, 3.413098217415098)]


tensor([0.5858, 3.4131])

In [63]:
loss_fn = nn.CrossEntropyLoss(reduction='mean', weight=class_weights)
loss_fn = loss_fn.cuda()

In [64]:
def train(model, Config, train_iter, optimizer, loss_fn):
    model.cuda()
    model.train()
    correct_curr, corrects, train_loss = 0, 0, 0
    recall_total = 0
    i = 0
    label_pos = 0
    print('total training batch = {}'.format(len(train_iter)))
    for batch in tqdm(train_iter):
        optimizer.zero_grad()
        x, label = batch
        x = x.cuda()
        label = label.cuda()
        batch_len = x.size()[0]
        
#         text1_emb = batch['pooler_output1']
#         text1_emb = text1_emb.cuda()
#         text2_emb = batch['pooler_output2']
#         text2_emb = text2_emb.cuda()
        
        logit = model(x)
        logit = torch.cat([1-logit, logit], dim=1)
        
#         loss_func = torch.nn.MSELoss()
#         loss_func = nn.NLLLoss()
#         loss_func = nn.CrossEntropyLoss()
        loss_func = loss_fn
        loss = loss_func(logit, label)
        loss.backward()
        optimizer.step()
        
        loss_curr = loss.item()
        if i%1000==0: print('loss_curr',loss_curr)
        train_loss += loss_curr
#         if i%1000==0: print('label',label)
#         if i%1000==0: print('logit',logit)
        correct_curr = (torch.argmax(logit,dim=1)==label).sum().item()
        corrects += correct_curr
        recall_total += ((torch.argmax(logit,dim=1)==label)&(label>0)).sum().item()
        label_pos += (label>0).sum()
        
#         text1_emb = text1_emb.to('cpu')
#         text2_emb = text2_emb.to('cpu')
        x = x.to('cpu')
        logit = logit.to('cpu')
        label = label.to('cpu')
        del loss 
        del x
        del logit
        torch.cuda.empty_cache()
        i += 1
        
#         if i % 1000 ==0 :
#             print('batch = {}, accuracy_curr = {:.2f}%, loss_curr = {}'.format(
#                 i, correct_curr/batch_len*100, loss_curr/batch_len))
            
        
    size = len(train_iter.dataset)
    train_loss /= size
    accuracy = 100.0 * corrects / size
    recall = 100.0 * recall_total / label_pos
    return train_loss, accuracy, recall

In [65]:
def val(model, Config, test_iter, loss_fn):
    model.cuda()
    model.eval()
    correct_curr, corrects, test_loss = 0, 0, 0
    recall_total = 0
    i = 0
    print('total testing batch = {}'.format(len(test_iter)))
    logit_all = []
    label_pos = 0
    for batch in tqdm(test_iter):
#         optimizer.zero_grad()
        x, label = batch
        x = x.cuda()
        label = label.cuda()
        batch_len = x.size()[0]
        
#         text1_emb = batch['pooler_output1']
#         text1_emb = text1_emb.cuda()
#         text2_emb = batch['pooler_output2']
#         text2_emb = text2_emb.cuda()
        
        logit = model(x)
        logit = torch.cat([1-logit, logit], dim=1)
        
        if len(logit_all)==0:
            logit_all = logit
        else:
            logit_all = torch.cat([logit_all, logit], dim=0)
        
#         loss_func = torch.nn.MSELoss()
#         loss_func = nn.NLLLoss()
#         loss_func = nn.CrossEntropyLoss()
        loss_func = loss_fn
        loss = loss_func(logit, label)
#         loss.backward()
#         optimizer.step()
        
        loss_curr = loss.item()
        if i%1000==0: print('loss_curr',loss_curr)
        test_loss += loss_curr
#         if i%1000==0: print('label',label)
#         if i%1000==0: print('logit',logit)
        correct_curr = (torch.argmax(logit,dim=1)==label).sum().item()
        corrects += correct_curr
        recall_total += ((torch.argmax(logit,dim=1)==label)&(label>0)).sum().item()
        label_pos += (label>0).sum()
        
#         text1_emb = text1_emb.to('cpu')
#         text2_emb = text2_emb.to('cpu')
        x = x.to('cpu')
        logit = logit.to('cpu')
        label = label.to('cpu')
        del loss 
        del x
        del logit
        torch.cuda.empty_cache()
        i += 1
        
#         if i % 1000 ==0 :
#             print('batch = {}, accuracy_curr = {:.2f}%, loss_curr = {}'.format(
#                 i, correct_curr/batch_len*100, loss_curr/batch_len))
            
        
    size = len(test_iter.dataset)
    test_loss /= size
    accuracy = 100.0 * corrects / size
    recall = 100.0 * recall_total / label_pos
    return test_loss, accuracy, recall, logit_all

In [66]:
best_accuracy = 0
CONFIG.epochs = 50
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
for epoch in range(CONFIG.epochs):
    print('Epoch {}'.format(epoch))
    train_loss, train_accuracy, train_recall = train(model=fm, Config=CONFIG, 
                                 train_iter=data_module.train_dataloader(), 
                                 optimizer=optimizer, loss_fn=loss_fn)
    scheduler.step()
#     test_loss, test_accuracy = valid(model=model_v1, Config=Config, test_iter=data_module.test_dataloader())
#     if epoch%10==0:
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {:.2f}% Recall: {:.2f}%'.format(
        epoch, train_loss, train_accuracy, train_recall))
    test_loss, test_accu, test_recall, test_res = val(
        model=fm, Config=CONFIG, test_iter=data_module.test_dataloader(), loss_fn=loss_fn)
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {:.2f}% Recall: {:.2f}%'.format(
        epoch, test_loss, test_accu, test_recall))
    if test_accu > best_accuracy:
        torch.save(fm.state_dict(), './fm-model/model_fm_'+str(CONFIG.name)+'_epoch'+str(epoch))

Epoch 0
total training batch = 5551


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


loss_curr 0.6931470036506653
loss_curr 0.6757427453994751
loss_curr 0.673324465751648
loss_curr 0.6809438467025757
loss_curr 0.6556721925735474
loss_curr 0.6800030469894409

Train Epoch: 0 	 Loss: 0.0016871425510895772 	 Accuracy: 51.58% Recall: 69.00%
total testing batch = 1388


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


loss_curr 0.6841935515403748
loss_curr 0.6623879671096802

Train Epoch: 0 	 Loss: 0.0016862901145820275 	 Accuracy: 56.17% Recall: 60.00%
Epoch 1
total training batch = 5551


...

loss_curr 0.6488986015319824
loss_curr 0.6627280116081238

Train Epoch: 47 	 Loss: 0.001654200446004792 	 Accuracy: 62.04% Recall: 60.00%
Epoch 48
total training batch = 5551


loss_curr 0.632979691028595
loss_curr 0.650076150894165
loss_curr 0.618922770023346
loss_curr 0.6548895239830017
loss_curr 0.638397216796875
loss_curr 0.6270387172698975

Train Epoch: 48 	 Loss: 0.0015869626768727075 	 Accuracy: 63.49% Recall: 69.00%
total testing batch = 1388


loss_curr 0.664377748966217
loss_curr 0.666667640209198

Train Epoch: 48 	 Loss: 0.0016542332099876312 	 Accuracy: 62.06% Recall: 60.00%
Epoch 49
total training batch = 5551


loss_curr 0.626829206943512
loss_curr 0.6571027636528015
loss_curr 0.640311062335968
loss_curr 0.6555317640304565
loss_curr 0.6502448916435242
loss_curr 0.6234210133552551

Train Epoch: 49 	 Loss: 0.0015869661700776427 	 Accuracy: 63.47% Recall: 69.00%
total testing batch = 1388


loss_curr 0.6725542545318604
loss_curr 0.6362221837043762

Train Epoch: 49 	 Loss: 0.0016540447001737104 	 Accuracy: 62.10% Recall: 60.00%
